<a href="https://colab.research.google.com/github/AriannaRampini/SpectralAdversarialAttacks/blob/main/run_adversarial_attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Universal Adversarial Examples on SMAL with PointNet classifier

First, we need to import the necessary modules.

In [1]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install openmesh

Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 7.9 MB 4.2 MB/s 
     |████████████████████████████████| 3.5 MB 20.8 MB/s 
     |████████████████████████████████| 2.3 MB 26.6 MB/s 
     |████████████████████████████████| 747 kB 24.4 MB/s 
     |████████████████████████████████| 325 kB 3.1 MB/s 
     |████████████████████████████████| 407 kB 41.1 MB/s 
     |████████████████████████████████| 41 kB 537 kB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.2-py3-none-any.whl size=535570 sha256=cb52ac097c30e2ba65740dc56f3372c23ea2feb51a51cd18a52cb5c0e20ac427
  Stored in directory: /root/.cache/pip/wheels/3f/08/13/2321517088bb2e95bfd0e45033bb9c923189e5b2078e0be4ef
Successfully built torch-geometric
     |████████████████████████████████| 9.1 MB 3.7 MB/s 
  Created wheel for openmesh: filename=openmesh-1.1.6-cp37-cp37m-linux_x86_64.whl size=607507 sha256=c1294ae9c2250a2895f1ba5124ba68f333e3dc38453b29253be7490c72f5ac

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
!git clone https://github.com/AriannaRampini/SpectralAdversarialAttacks.git

Cloning into 'SpectralAdversarialAttacks'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 95 (delta 1), reused 5 (delta 1), pack-reused 86
Unpacking objects: 100% (95/95), done.


In [4]:
## Built-in libraries.
import sys
import os 

## Third party libraries.
import matplotlib.pyplot as plt 
import numpy as np
import torch 

REPO_ROOT  = '/content/SpectralAdversarialAttacks'
DEVICE     = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
SRC_DIR    = os.path.join(REPO_ROOT, "src")

## Repository modules.
sys.path.insert(0, SRC_DIR)
import models
import dataset
from utils import visualize, compare, visualize_and_compare

Load dataset SMAL and a pre-trained PointNet classifier:

In [5]:
from models.pointnet import *

## Pre-trained model weights.
SMAL_POINTNET = os.path.join(REPO_ROOT, "model_data/smal-pointnet.pt")

# Encoder.
LATENT_SPACE = 128
ENC = SimplePointNet(latent_dimensionality=LATENT_SPACE,
                                     convolutional_output_dim=512,
                                     conv_layer_sizes=[32, 128, 256],
                                     fc_layer_sizes=[512, 256, 128],
                                     transformer_positions=[0]).to(DEVICE)

# Classifier.
CLA = nn.Sequential(nn.Linear(LATENT_SPACE, 64), nn.ReLU(), nn.Linear(64,5)).to(DEVICE)

## Model.
model = SMALClassifier(ENC, CLA, LATENT_SPACE).to(DEVICE)
model.load_state_dict(torch.load(SMAL_POINTNET, map_location=torch.device("cpu")))

<All keys matched successfully>

Load a set of shapes from SMAL dataset:

In [6]:
## SMAL dataset.
SMAL = os.path.join(REPO_ROOT, "datasets/smal")

label_dic = {0: 'lions',
             1: 'cows',
             2: 'dogs',
             3: 'hippos',
             4: 'horses'}

custom_list = [1,2,3,4,5]
customdata  = dataset.SmalDataset(SMAL, device=DEVICE, train=False, test=False, custom=True, custom_list=custom_list, transform_data=False)
n_shapes    = len(customdata)

Processing...
100%|██████████| 21/21 [00:00<00:00, 70.65it/s]
Done!


## Configure adversarial attack

In [12]:
from universal.builder import Builder

## Parameters used to search the adversarial sample.
params = {
    Builder.USETQDM    : 'notebook',
    Builder.MIN_IT     : 500,   #<- Number of iterations for the adversarial example computation (common value: 1000).
    Builder.LEARN_RATE : 1e-4,   #<- Learning rate used during optimization.
    # Coefficients.
    Builder.ADV_COEFF  : 0.07,   #<- Starting coefficient applied to the adversarial loss (updated via exponential search).
    Builder.REG_COEFF  : 0,      #<- Coefficient applied to the regularization term.
    Builder.SIM_COEFF  : 0,      #<- Coefficient applied to the cross similarity term (gamma).
    Builder.ISO_COEFF  : 10000,  #<- Coefficient applied to the isospectralization term.
    Builder.ADV_LOSS_K : 0.5,
    # Eigenvalues number.
    Builder.EIGS_ISOSP_NUM     : 61, #<- Number of eigenvalues for isospectralization.
    Builder.EIGS_BANDWIDTH_NUM : 20  #<- Number of eigenvalues for low bandwith distorsion.
}


## Configure adversarial example components using builder.
#--------------------------------------------------------
builder = Builder(search_iterations=1)
builder.set_classifier(model)
builder.set_dataset(customdata)
builder.set_target(None)
builder.set_template_index(0)

## Set perturbation.
from universal.perturbation import UniversalPerturbation
builder.set_perturbation(UniversalPerturbation)

## Set losses.
from universal.loss import *
# Similarity Losses.
builder.set_similarity_loss(ZeroLoss)
builder.set_cross_similarity_loss(ZeroLoss)
# Adversarial Loss.
builder.set_adversarial_loss(UniversalAdversarialLoss)
# Isospectralization loss.
builder.set_isospectralization_loss(UniversalIsospec)
# Regularization Loss
builder.set_regularization_loss(ZeroLoss)

## Perform adversarial attack

In [13]:
adv_ex = builder.build(**params)

[0,0.07] ; c=0.07


  0%|          | 0/500 [00:00<?, ?it/s]

## Visualize results

Check misclassification:

In [14]:
## Print original and final logits and compute success list.

cmodel = lambda x : model(x.float())
success_list = []

for shape_index in range(len(customdata)):
    
    print('\nShape ', custom_list[shape_index])    

    perturbed_pos_i = adv_ex.perturbed_positions_i(shape_index)
    out: torch.Tensor = cmodel(perturbed_pos_i)
    _, lbl_prediction = out.max(dim=-1)
    print('Perturbed logits : ', out.detach().cpu().numpy())

    pos_i = customdata[shape_index].pos
    out: torch.Tensor = cmodel(pos_i)
    _, lbl_original = out.max(dim=-1)
    print('Original logits  : ', out.detach().cpu().numpy())

    success = (lbl_prediction != lbl_original).item()
    success_list.append(success)

    print("Successful" if not int(lbl_prediction) == int(lbl_original) else "Unsuccessful",
          "[prediction: ", int(lbl_prediction),
          ", original: ", int(lbl_original), "]")


Shape  1
Perturbed logits :  [-1.7173573  -0.69079316 -2.8721423   0.65419143  0.0826599 ]
Original logits  :  [-2.4567692  -0.55528176 -3.6982324  -0.3219935   0.9267939 ]
Successful [prediction:  3 , original:  4 ]

Shape  2
Perturbed logits :  [-2.5818388  -0.95214045 -3.8883982   0.9711925   0.34388533]
Original logits  :  [-5.9103875  0.4654911 -7.321789  -5.8181086  4.6407466]
Successful [prediction:  3 , original:  4 ]

Shape  3
Perturbed logits :  [-2.8496006  1.618039  -3.7168736 -2.4125614  1.2957389]
Original logits  :  [-3.1061332  0.0154103 -4.0390096 -1.1888632  1.5800855]
Successful [prediction:  1 , original:  4 ]

Shape  4
Perturbed logits :  [ 0.23207596 -0.12300321 -2.052194   -0.9405824  -0.30796948]
Original logits  :  [-4.669506  0.562531 -6.034157 -5.160282  3.748162]
Successful [prediction:  0 , original:  4 ]

Shape  5
Perturbed logits :  [-1.7109891  -0.7933625  -2.9664404   0.7173231   0.04025823]
Original logits  :  [-2.6773639  -0.28105524 -3.9196918  -1.3

Visualize an adversarial shape:

In [15]:
shape_index = 0 # Shape to visualize.

k_evals = params[Builder.EIGS_ISOSP_NUM]

## Original shape.
pos = customdata[shape_index].pos
faces = adv_ex.faces(shape_index)

## Load perturbed shape.
perturbed_pos = adv_ex.perturbed_positions_i(shape_index)

## Rotate meshes for visualization.
pos_i_rotated           = pos.clone()
pos_i_rotated[:,1]      = -1 * pos_i_rotated[:,1]

perturbed_pos_i_rotated      = perturbed_pos.clone()
perturbed_pos_i_rotated[:,1] = -1 * perturbed_pos_i_rotated[:,1]

print(success_list[shape_index])
visualize_and_compare(perturbed_pos_i_rotated, faces, pos_i_rotated, faces, (pos-perturbed_pos).norm(p=2,dim=-1))

True
